# 理論式を用いたインダクタンスの算出

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os
import scipy                                                                                                                           
import math
import scipy.optimize as opt
import sympy as sy

### <span style="color: red; ">変数指定</span>

所望のインダクタンス$L_o$, シミュレーションを回す変数範囲

In [41]:
# 所望のインダクタンスL_o(uH)
Lo = 500
# 外径(単位mm)
Dout = 30
# 周波数f0
f0 = 200*10**(3)

# wは単位mm, 範囲指定は線形スケールで行う
w_start = 0.1
w_end = 5
w_step = 0.1

# wは単位mm, 範囲指定は線形スケールで行う
s_start = 0.1
s_end = 5
s_step = 0.1

しみゅれーしょんを回す配列準備

In [4]:
# 浮動少数点数型の変数の配列
w_values = list(np.arange(w_start, w_end, w_step))
s_values = list(np.arange(s_start, s_end, s_step))

矩形コイルのインダクタンスの理論式
$$
L(nH) = 1.62\times10^{-3}\times{d_{out}}^{-1.21}w^{-0.147}d_{avg}^{2.40}n^{1.78}s^{-0.030}
$$
に従って算出する。
なお、各変数の単位はL(nH), Dout, w, Davg, sはそれぞれumである。

In [5]:
# 各種係数
# 矩形コイルインダクタンス理論式
B = 1.62*(10**(-3))
a_dout = -1.21
a_w = -0.147
a_davg = 2.40
a_n = 1.78
a_s = -0.030

# 抵抗率理論式
sigma = 1.74*10**(-8) # 銅の抵抗率
t = 35 # コイル銅箔厚さ(um)

ラグランジュの未定乗数法での数値解析は現実的でない解に関しての解析を行っている

→ありうる範囲で全探索する. 各パターンで$L$と$R_L$を計算し, 目的の$L_o$の誤差5％以内のものから最も$R_L$が小さいものを選ぶ

コイルの抵抗$R_L$は以下の式で表される(t, σは定数, lengthはn, w, sの関数)
$$
R_L(n, w, s) = \frac{\sigma}{tw}\times length(n, w, s)
$$
インダクタンス$L$は、
$$
L(n, w, s) = 1.62\times10^{-3}\times{d_{out}}^{-1.21}w^{-0.147}d_{avg}^{2.40}n^{1.78}s^{-0.030}
$$

(注: まだlengthは正確な定義まで練ってない, ひとまずこれくらいの値)
$$
length= 4(D_{out} -2n(s+w)-w)
$$

変数定義

In [6]:
w, s, n, Din, Dav, h = sy.symbols("w s n Din Dav h")

In [7]:
length = (Dout - 2*n*(s+w)-w)*4 # コイルの総長さ(mm)
R_L= (sigma*length*10**(-3))/((w*10**(-3))*(t*10**(-3)))

Lを定義

(式内で$D_{out}$, $D_{av}$, w, sを$10^3$倍しているのは, コイルの理論式ではumなのに対し, 指定はmmだから)

In [8]:
Din=Dout-w*(2*n)-s*(2*n-1)
Dav=(Dout+Din)/2
L = B*((Dout*10**3)**a_dout)*((w*10**3)**a_w)*((Dav*10**3)**a_davg)*(n**a_n)*((s*10**3)**a_s)

w, sに応じて、巻き数の上限は決定する
$$
n_{max} = \frac{D_{out}+s}{2(s+w)}
$$

In [9]:
nmax = (Dout+s)/(2*(s+w))

収集するデータのDataFrame用のカラムを準備

In [35]:
L_str = '$L(\mu H)$'
R_str = '$R_L(\mu \Omega)$'
C_str = '$C(\mu F)$'

columns = ['w','s', 'n', L_str, R_str]

In [36]:
sim_df = pd.DataFrame(columns= columns)

In [37]:
for W in w_values:
    for S in s_values:
        n_m = nmax.subs([(w, W),(s, S)])//1
        for N in range(1,n_m):
            sim_df=sim_df.append({'w': W,'s': S, 'n': N, L_str: float(L.subs([(w, W),(s, S),(n, N)])*10**(-3)),R_str: float((10**6)*R_L.subs([(w, W),(s, S),(n,N)]))}, ignore_index=True)
            

$$
\omega_{0}=\frac{1}{\sqrt{LC}}
$$
より
$$
C=\frac{1}{L\omega_{0}^2}=\frac{1}{4\pi^2f_{0}^2L}
$$

In [38]:
sim_df[C_str]=(10**(6))/(4*((np.pi)**2)*(f0**2)*(sim_df[L_str]*(10**(-6))))
sim_df.sort_values(L_str)

,w,s,n,$L(\mu H)$,$R_L(\mu \Omega)$,$C(\mu F)$
6568,4.9,3.4,1.0,0.042649,3.449563,14.847971
6533,4.8,3.6,1.0,0.042706,3.480000,14.828440
6567,4.9,3.3,1.0,0.042907,3.530729,14.758879
6532,4.8,3.5,1.0,0.042961,3.562857,14.740209
6495,4.7,3.7,1.0,0.043023,3.596353,14.719179
...,...,...,...,...,...,...
65,0.1,0.1,66.0,66.195166,69.600000,0.009567
61,0.1,0.1,62.0,66.197686,101.417143,0.009566
64,0.1,0.1,65.0,66.270858,77.554286,0.009556
62,0.1,0.1,63.0,66.272524,93.462857,0.009555


In [39]:
sim_df.to_csv('coil_list.csv')